<a href="https://colab.research.google.com/github/alirezash97/Pattern-Recognition-Course/blob/main/Cervical_Cancer/Cervical_Cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00383/risk_factors_cervical_cancer.csv'

--2021-03-07 10:50:09--  https://archive.ics.uci.edu/ml/machine-learning-databases/00383/risk_factors_cervical_cancer.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102059 (100K) [application/x-httpd-php]
Saving to: ‘risk_factors_cervical_cancer.csv’

risk_factors_cervic 100%[===================>]  99.67K   621KB/s    in 0.2s    

2021-03-07 10:50:09 (621 KB/s) - ‘risk_factors_cervical_cancer.csv’ saved [102059/102059]



In [1]:
import pandas as pd 
import numpy as np


dataset = pd.read_csv('/content/risk_factors_cervical_cancer.csv')
dataset.head()

,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,IUD (years),STDs,STDs (number),STDs:condylomatosis,STDs:cervical condylomatosis,STDs:vaginal condylomatosis,STDs:vulvo-perineal condylomatosis,STDs:syphilis,STDs:pelvic inflammatory disease,STDs:genital herpes,STDs:molluscum contagiosum,STDs:AIDS,STDs:HIV,STDs:Hepatitis B,STDs:HPV,STDs: Number of diagnosis,STDs: Time since first diagnosis,STDs: Time since last diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Hinselmann,Schiller,Citology,Biopsy
0,18,4.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,?,?,0,0,0,0,0,0,0,0
1,15,1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,?,?,0,0,0,0,0,0,0,0
2,34,1.0,?,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,?,?,0,0,0,0,0,0,0,0
3,52,5.0,16.0,4.0,1.0,37.0,37.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,?,?,1,0,1,0,0,0,0,0
4,46,3.0,21.0,4.0,0.0,0.0,0.0,1.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,?,?,0,0,0,0,0,0,0,0


In [2]:
# check for missing values 

dataset = dataset.replace(['?'], np.nan)
dataset.isna().sum()

Age                                     0
Number of sexual partners              26
First sexual intercourse                7
Num of pregnancies                     56
Smokes                                 13
Smokes (years)                         13
Smokes (packs/year)                    13
Hormonal Contraceptives               108
Hormonal Contraceptives (years)       108
IUD                                   117
IUD (years)                           117
STDs                                  105
STDs (number)                         105
STDs:condylomatosis                   105
STDs:cervical condylomatosis          105
STDs:vaginal condylomatosis           105
STDs:vulvo-perineal condylomatosis    105
STDs:syphilis                         105
STDs:pelvic inflammatory disease      105
STDs:genital herpes                   105
STDs:molluscum contagiosum            105
STDs:AIDS                             105
STDs:HIV                              105
STDs:Hepatitis B                  

In [3]:
# get missing values indexes

missing_values_indexes = {}
for column in dataset:
  if dataset[column].isnull().values.any():
    missing_values_indexes[column] = dataset[column].isnull().to_numpy().nonzero()

In [4]:
def missing_handle_classmean(dataframe, column, indexes):
  

  for index in indexes[0]:
    
    missing_value_class_values = []
  
    
    index_label = np.array([dataframe['Hinselmann'][index], dataframe['Schiller'][index], dataframe['Citology'][index], dataframe['Biopsy'][index] ]) 
    for row_index, row in dataset.iterrows():

      row_label = np.array([dataframe['Hinselmann'][row_index], dataframe['Schiller'][row_index], dataframe['Citology'][row_index], dataframe['Biopsy'][row_index] ])
      if str(index_label) == str(row_label):
        if (dataframe[column][row_index] is None) or (str(dataframe[column][row_index]) == 'nan'):
          pass
        else:
          missing_value_class_values.append( int(float(dataframe[column][row_index])) )

    if missing_value_class_values:
      dataframe[column][index] = np.mean(missing_value_class_values)
    else:
      dataframe[column][index] = 0

  
  return dataframe

In [23]:
def missing_handle_mean(dataframe, column, indexes):
  
  for index in indexes[0]:

    
    column_without_nan = dataframe[column].dropna()
    column_without_nan = np.array(np.array(column_without_nan.values, dtype=float), dtype=int)
    dataframe[column][index] = column_without_nan.mean()
  
  return dataframe

In [24]:
def missing_handler(dataframe, missing_values_indexes, method='Class_mean'):
  

  if method=='Class_mean':
    for key in missing_values_indexes:
      dataframe = missing_handle_classmean(dataframe, key, missing_values_indexes[key])

  elif method=='feature_mean':
    for key in missing_values_indexes:
      dataframe = missing_handle_mean(dataframe, key, missing_values_indexes[key])
  else:
    pass

  
  return dataframe




In [25]:
dataset = missing_handler(dataset, missing_values_indexes, method='feature_mean')
dataset.isna().sum()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Age                                   0
Number of sexual partners             0
First sexual intercourse              0
Num of pregnancies                    0
Smokes                                0
Smokes (years)                        0
Smokes (packs/year)                   0
Hormonal Contraceptives               0
Hormonal Contraceptives (years)       0
IUD                                   0
IUD (years)                           0
STDs                                  0
STDs (number)                         0
STDs:condylomatosis                   0
STDs:cervical condylomatosis          0
STDs:vaginal condylomatosis           0
STDs:vulvo-perineal condylomatosis    0
STDs:syphilis                         0
STDs:pelvic inflammatory disease      0
STDs:genital herpes                   0
STDs:molluscum contagiosum            0
STDs:AIDS                             0
STDs:HIV                              0
STDs:Hepatitis B                      0
STDs:HPV                              0


In [26]:
def split_dataset(df, train_percentage=0.8, test_percentage=0.2):



  train = df.sample(frac=(1-test_percentage),random_state=200) #random state is a seed value
  test = df.drop(train.index)

  return train, test

In [27]:
trainset, testset = split_dataset(dataset, 0.8, 0.2)

In [28]:
print(len(trainset))
print(len(testset))

686
172


In [29]:
# dataframe to numpy array 
x_train = trainset.drop(['Hinselmann', 'Schiller', 'Citology', 'Biopsy'], axis = 1).values
y_train = trainset.loc[:, ['Hinselmann', 'Schiller', 'Citology', 'Biopsy']].values.astype('int')

x_test = testset.drop(['Hinselmann', 'Schiller', 'Citology', 'Biopsy'], axis = 1).values
y_test = testset.loc[:, ['Hinselmann', 'Schiller', 'Citology', 'Biopsy']].values.astype('int')

In [30]:
print('x_train shapes: ', x_train.shape)
print('y_train shapes: ', y_train.shape)
print('x_test shapes: ', x_test.shape)
print('y_test shapes: ', y_test.shape)

x_train shapes:  (686, 32)
y_train shapes:  (686, 4)
x_test shapes:  (172, 32)
y_test shapes:  (172, 4)


In [31]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [32]:
print(clf.predict(x_test))
print('--------------------')
print(y_test)

[[1 1 1 1]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 1 0 1]
 [0 0 0 0]
 [0 0 0 0]
 [1 1 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [1 1 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 1 0 1]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 1 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [1 1 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [1 1 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [1 1 0 1]
 [1 1 1 1]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [1 1 1 1]
 [0 0 0 0]
 [0 0 0 0]
 [1 1 0 0]
 [1 1 1 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]

In [33]:
clf.score(x_test, y_test)

0.7848837209302325

In [ ]:
# as you can see, accuracy score for this model is 0.78( using the mean of the column as missing value handler )
# but if we use the mean of the column "by class" the accuracy score will be over 0.96!